In [81]:
import pandas as pd
import re
from pathlib import Path

BASEDIR = Path.cwd()

df = pd.read_excel(BASEDIR / 'form responses.xlsx')
df['TRIBUNAL'] = 'MG'
df.columns = [i.strip() for i in df.columns]

#######
df_codebook = pd.read_excel(BASEDIR / 'codebook.xlsx')
df_codebook = df_codebook.dropna(subset=['resposta'])
def edit(x):
  a,b = x
  if pd.isna(b): return a
  return b

df_codebook['id_def'] = df_codebook[['respostaId', 'equivalente']].apply(edit, axis=1)
name_to_id = {a:b for _,a,b in df_codebook[['resposta', 'id_def']].itertuples()}
id_to_name = {b:a for _,a,b in df_codebook[['resposta', 'respostaId']].itertuples()}
respId_to_respText = {a:b for _, a,b in df_codebook[['respostaId', 'resposta']].itertuples()}

#######
df_columns = pd.read_excel(BASEDIR / 'columns.xlsx')
MULTIPLE = df_columns[df_columns.tipo == 'multiple'].valor.astype(int).unique()
DCOLS = {a:b for _,a,b in df_columns[['nome', 'valor']].itertuples()}
colId_to_colName = {int(a):f'[{b}][{int(a):02}]' for _, a, b in df_questions[[ 'valor', 'grupo', ]].itertuples()}
########
df_treatments = pd.read_excel(BASEDIR / 'treatments.xlsx')
df_treatments = df_treatments[['RESPOSTA','b- nomes separados ']].rename({'RESPOSTA': 'resposta', 'b- nomes separados ': 'classificacao'}, axis=1)

########
df_translations = pd.read_excel(BASEDIR / 'translations.xlsx')
df_translations = df_translations[['original', 'Tradução em Inglês']].dropna()

########
df_deseases = pd.read_excel(BASEDIR / 'deseases.xlsx')
df_deseases.RESPOSTA.apply(lambda x : x.strip('"').replace('\n', ' '))
BOX = []
for ind in range(1, 8):
  dfp = df_deseases[['RESPOSTA', f'classificacao_0{ind}']]
  dfp.columns = ['resposta', 'classificacao']
  BOX.append(dfp)
df_deseases = pd.concat(BOX).dropna().applymap(lambda x : x.strip().replace('\n', ' '))
df_deseases.classificacao = df_deseases.classificacao.replace(name_to_id)


# sanitize operations
df = df.rename(DCOLS, axis=1)
df = df.drop([i for i in df.columns if 'Unn' in str(i)], axis=1)
df = df.drop([int(i) for i in df.columns if i < 0 and i != -4 and i !=-2], axis=1)
_ = pd.Series(df.columns).value_counts()
for c in _[_>1].keys():
  X = df[c].apply(lambda x : x.dropna(), axis=1)
  df = df.drop([c], axis=1)
  df[c] = X
crit = re.compile('^\D+|\.0$')
df[-4] = df[-4].apply(lambda x : crit.sub('', str(x)))
df = df[df[-4].apply(lambda x : x.isnumeric())]
def edit(x):
  a,b = x
  return f'{a}{b}'
df['id'] = df[[52, -4]].apply(edit, axis=1)
df['data'] = df[-2].apply(lambda x : pd.to_datetime(x, dayfirst=True))
df = df.sort_values(by=['data'], ascending=False)
df = df.drop_duplicates(subset=['id'], keep='first')
df = df.drop([-4, -2], axis=1)


#########
COLS = [i for i in df.columns if i not in ['id', 'data', 52] + [41, 42, 43, 44, 46, 17, 20, 25, 47,]]
BOX = []

for c in COLS:
  dfp = df[['id', 52, c]]

  if c == 14:

    dfp = df[['id', 52, c]]

    dfp2 = dfp.dropna().assign(pergunta=int(c))
    dfp2.columns = ['id', 'TRIBUNAL', 'RESPOSTA', 'PERGUNTA',]
    dfp3 = pd.merge(dfp2, df_treatments, left_on='RESPOSTA', right_on='resposta', how='left').rename({'classificacao': 'ALTERNATIVA'}, axis=1).drop(['resposta'], axis=1)
    dfp3.RESPOSTA = True
    dfp3.ALTERNATIVA = dfp3.ALTERNATIVA.str.strip()
    dfp4 = pd.merge(dfp3, df_translations, left_on='ALTERNATIVA', right_on='original', how='outer').dropna(subset=['ALTERNATIVA']).dropna()
    dfp5 = dfp4.drop(['ALTERNATIVA', 'original'], axis=1).rename({'Tradução em Inglês': 'ALTERNATIVA' }, axis=1)
    BOX.append(dfp5)

  elif c in MULTIPLE:

    dfp2 = dfp.dropna(subset=[c])
    dfp2[c] = dfp2[c].apply(lambda x : x.split(','))
    dfp3 = dfp2.explode(c)
    dfp3[c] = dfp3[c].apply(lambda x : x.strip()).apply(lambda x : x.strip('"').replace('\n', ' '))

    dfp3.columns = ['id', 'TRIBUNAL', 'ALTERNATIVA']
    dfp3['RESPOSTA'] = True
    dfp4 = dfp3.drop_duplicates()
    dfp4['PERGUNTA'] = int(c)
    if c == 12:
      dfp4 = pd.merge(dfp4, df_deseases, left_on='ALTERNATIVA', right_on='resposta', how='left').drop(['ALTERNATIVA', 'resposta'], axis=1).rename({'classificacao': 'ALTERNATIVA'}, axis=1)

    BOX.append(dfp4.dropna())
  else:
    dfp2 = dfp.assign(pergunta=int(c), alternativa=None)
    dfp2.columns = ['id', 'TRIBUNAL', 'RESPOSTA', 'PERGUNTA', 'ALTERNATIVA']
    dfp2.RESPOSTA = dfp2.RESPOSTA.replace(name_to_id)
    dfp3 = dfp2.drop_duplicates()
    dfp3.ALTERNATIVA = dfp3.ALTERNATIVA.fillna('unique')
    BOX.append(dfp3.dropna())

df3 = pd.concat(BOX)
df3 = df3.applymap(lambda x : x.strip('"').replace('\n', ' ') if isinstance(x,str) else x)
df3 = df3.replace(name_to_id)

df3 = df3[df3.ALTERNATIVA!=-1]
df3 = df3[df3.RESPOSTA!=-1]

df3 = df3[~df3.ALTERNATIVA.apply(lambda x : isinstance(x, str) and x != 'unique')]
df3 = df3[df3.PERGUNTA < 50]

df3.RESPOSTA = df3.RESPOSTA.astype(int)
df3.RESPOSTA = df3.RESPOSTA.astype(str)
df4 = pd.pivot_table(df3,
               index=[ 'TRIBUNAL', 'id'],
               columns=['PERGUNTA', 'ALTERNATIVA'],
               values=['RESPOSTA'],
               aggfunc=lambda x : x.unique()[0],
               )

def edit_cols(x):
  a,b,c = x
  if c == 'unique': return f'{colId_to_colName[b]}[multiple_choice]'
  return f'{colId_to_colName[b]}[{int(c):04}]'

df4.columns = [edit_cols(i) for i in df4.columns]

df5 = df4.reset_index()
crit = re.compile('\[.+?\]')
for c in df5.columns:
  if 'multiple_choice' in c:
    df5[c] = df5[c].astype(int, errors='ignore').replace(id_to_name).str.replace(crit, '').str.strip()